In [ ]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
filepath = "/Users/balqeesjabri/Downloads/titanic.csv"
titanic_data = pd.read_csv(filepath)

In [ ]:
titanic_data.shape

In [ ]:
titanic_data.head()

In [ ]:
titanic_data.info()

we can conclude that Cabin has the most missing values followed by Age and embarked. However, there can still be other ways that missing data can be hiding in the data. For example,object data types.

In [ ]:
titanic_data.columns

In [ ]:
# Separate variables into numerical and categorical
numerical_vars = titanic_data.select_dtypes(include=['int64', 'float64']).columns
categorical_vars = titanic_data.select_dtypes(include=['object']).columns
# Display the number of numerical and categorical variables
print(f"Number of Numerical variables: {len(numerical_vars)}")
print(f"Number of Categorical variables: {len(categorical_vars)}")

In [ ]:
# Statistical summary of numerical variables
titanic_data[numerical_vars].describe()

The descriptive statistics of the numerical variables give us the following insights:

PassengerId: The average PassengerId is around 446, with a minimum of 1 and a maximum of 891. The standard deviation is approximately 257.35

Survived: The average of Survivors is around 0.38.The standard deviation is around 0.4866.

Age: The average Age is 29.7, but the standard deviation is 14.5.

Fare: The average overall Fare is about 32.2, with a standard deviation of approximately 49.69.



In [ ]:
y = titanic_data['Survived']  # Add the target variable as a new column

# Create the pair plot with 'hue' as the target variable
sns.pairplot(titanic_data.drop(columns='PassengerId'), hue='Survived', markers=["o", "s"], palette='tab10')

# Set the title for the pair plot
plt.suptitle("Pair Plot with Hue for Target Variable", y=1.02)

# Show the plot
plt.show()

the reason why categorical values are not showing becuase most plotting libraries, including Seaborn and Matplotlib, require numerical data for creating visualizations. 

In [ ]:
categorical_features = titanic_data.select_dtypes(include=['object']).drop(columns='Name')
# Plot each categorical feature separately

# Loop through each categorical feature
for feature in categorical_features:
    # Get the top N most frequent categories
    top_n_categories = categorical_features[feature].value_counts().nlargest(5)


    # Plot the bar plot for the top N categories
    plt.figure(figsize=(4, 2))
    sns.barplot(x=top_n_categories.index, y=top_n_categories.values)
    plt.title(f'Top {5} {feature} Categories')
    plt.xlabel(f'{feature} Category')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# # Check for missing values in numerical variables
num_missing_values = titanic_data[numerical_vars].isnull().sum()
num_missing_values = num_missing_values[num_missing_values > 0]
# Calculate the percentage of missing values
num_missing_percent = num_missing_values / len(titanic_data) * 100

# Create a DataFrame to display the results
num_missing_df = pd.DataFrame({'Missing Values': num_missing_values, 'Percentage': num_missing_percent})
num_missing_df.sort_values(by='Percentage', ascending=False)

The table above lists numerical variables that contain missing values along with the percentage of missing data for each.

Age: Approximately 19.86532% of the data is missing

In [ ]:
# Count unique values in categorical variables
cat_unique_counts = titanic_data[categorical_vars].nunique().sort_values(ascending=False)

# Check for missing values in categorical variables
cat_missing_values = titanic_data[categorical_vars].isnull().sum()
cat_missing_values = cat_missing_values[cat_missing_values > 0]

# Calculate the percentage of missing values
cat_missing_percent = cat_missing_values / len(titanic_data) * 100

# Create a DataFrame to display the results
cat_missing_df = pd.DataFrame({'Unique Values': cat_unique_counts, 'Missing Values': cat_missing_values, 'Percentage': cat_missing_percent})
cat_missing_df.sort_values(by='Missing Values', ascending=False)

The table above lists categorical variables, the count of unique values, missing values, and the percentage of missing data for each.

Cabin has a high percentage of missing values while Embarked has a low percentage. 

In [ ]:
# CORRELATION MATRIX
numerical_features = titanic_data.select_dtypes(include=['int64', 'float64']).drop(columns='PassengerId')
# Compute the correlation matrix
correlation_matrix = numerical_features.corr()

# Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(data=correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix Heatmap")
plt.show()

Correlation with Target: Pclass and Age
Pairwise Correlations: Fare and Pclass, Age and Pclass

There is no set value for what counts as “highly correlated”, however a general rule is a correlation of 0.7 (or -0.7). There are no pairs of features with a correlation of 0.7 or higher, so we do not need to consider leaving any features out of our model based on multicollinearity.

We have classification problem so we need to check the balance of the given data.

In [ ]:
# 1. CLASS DISTRIBUTION 
class_distribution = y.value_counts()
print(class_distribution)

In [ ]:
# Class Proportions: Calculate the proportion of samples in each class relative to the total number of samples.
# If any class has a significantly smaller proportion, it indicates an imbalanced dataset.
class_proportions = y.value_counts(normalize=True)
print(class_proportions*100)

In [ ]:
# Bar Plot: Visualize the class distribution using a bar plot. 
plt.figure(figsize=(8, 5))
sns.countplot(y)
plt.title('Class Distribution')
plt.xlabel('Target Class')
plt.ylabel('Count')
plt.show()

this plot indicates that my dataset is highly imbalanced, it is possible that the "survived" class has very few samples, leading to the appearance of a single bar with the value zero on the x-axis. In such cases, the bar representing the minority class (survived) might be too small to be visible in the plot.

In [ ]:
# Bar Plot: Visualize the class distribution using a horizontal bar plot.
plt.figure(figsize=(8, 5))
sns.countplot(x=y)  # Use x parameter instead of y
plt.title('Class Distribution')
plt.xlabel('Count')
plt.ylabel('Target Class')  # Switch the axis labels
plt.show()


In [ ]:
# Class Imbalance Ratio: Calculate the imbalance ratio,
# which is the ratio of the size of the majority class to the size of the minority class.

imbalance_ratio = class_distribution.max() / class_distribution.min()
print("Imbalance Ratio:", imbalance_ratio)

since the ratio is more than one, it suggests the dateset is imbalanced

 There is imbalance between two classes (approx 28% difference).So consider adapting performance metrics like "recall", "precision" and "f1 score" instead of "accurary" to assess the ML model. 

In [ ]:
# RESAMPLING TECHNIQUES TO SOLVE CLASS IMBALANCE 
# Separating the target that will be 0 and class 1.
# survivor_count_0, survivor_count_1 = titanic_data['Survived'].value_counts()

# Separate target
survivor_0 = titanic_data[titanic_data['Survived'] == 0]
survivor_1 = titanic_data[titanic_data['Survived'] == 1]# print the shape of the class
print('Survived 0:', survivor_0.shape)
print('Survived 1:', survivor_1.shape)

# target count 
survivor_count_0 = len(survivor_0)
survivor_count_1 = len(survivor_1)

In [ ]:
# Random Under-Sampling: removing some observations of the majority class.
# This is done until the majority and minority class is balanced out.
# can be a good choice when you have a ton of data -think millions of rows.
# But a drawback to undersampling is that we are removing information that may be valuable.

""" This line randomly samples a subset of samples from the majority class (Survived=0) so that the number of samples in survivor_0_under
    becomes equal to the number of samples in the minority class (Survived=1).
    This is the under-sampling step, which creates a balanced dataset with an equal number of samples from both classes. """
# Calculate the count of samples in each class


survivor_0_under = survivor_0.sample(survivor_count_1)

test_under = pd.concat([survivor_0_under, survivor_1], axis=0)

print("total survive of 1 and0:",test_under['Survived'].value_counts())# plot the count after under-sampeling
test_under['Survived'].value_counts().plot(kind='bar', title='count (target)')

Random Over-Sampling
""" Oversampling can be defined as adding more copies to the minority class.
    Oversampling can be a good choice when you don’t have a ton of data to work with.
    A con to consider is that it can cause overfitting and poor generalization to your test set. """


In [ ]:

# Perform over-sampling on the minority class (Survived=1)
survivor_1_over = survivor_1.sample(survivor_count_0, replace=True)

# Concatenate the over-sampled minority class with the majority class
test_over = pd.concat([survivor_1_over, survivor_0], axis=0)

# Print the total number of samples for each class after over-sampling
print("total survivors of 1 and 0:", test_over['Survived'].value_counts())

# Plot the count after over-sampling
test_over['Survived'].value_counts().plot(kind='bar', title='count (target)')

plt.show()


How to Balance Data With the Imbalanced-Learn Python Module?
A number of more sophisticated resampling techniques have been proposed in the scientific literature.

For example, we can cluster the records of the majority class and do the under-sampling by removing records from each cluster, thus seeking to preserve information. In over-sampling, instead of creating exact copies of the minority class records, we can introduce small variations into those copies, creating more diverse synthetic samples.

In [ ]:
from collections import Counter
import imblearn
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42, replacement=True)# fit predictor and target variable
X = titanic_data.drop(columns='Survived')
y = titanic_data['Survived']
X_rus, y_rus = rus.fit_resample(X, y)

print('original dataset shape:', Counter(y))
print('Resample dataset shape', Counter(y_rus))


Random Over-Sampling With imblearn
One way to fight imbalanced data is to generate new samples in the minority classes. The most naive strategy is to generate new samples by random sampling with the replacement of the currently available samples. The RandomOverSampler offers such a scheme.

In [ ]:
# import library
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X = titanic_data.drop(columns='Survived')
y = titanic_data['Survived']
# fit predictor and target variablex_ros, y_ros = ros.fit_resample(x, y)
X_ros, y_ros = ros.fit_resample(X, y)

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))

Under-Sampling: Tomek Links
Tomek links are pairs of very close instances but of opposite classes. Removing the instances of the majority class of each pair increases the space between the two classes, facilitating the classification process.

In [ ]:
# import library
from imblearn.under_sampling import TomekLinks

tl = TomekLinks()

# fit predictor and target variable
X_tl, y_tl = tl.fit_resample(X, y)

print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_tl))

In [ ]:


from sklearn.model_selection import train_test_split

# # Assuming 'test_under' is the balanced DataFrame containing both classes
# X = test_over.drop('Survived', axis=1)  # Features (all columns except 'Survived')
# y = test_over['Survived']  # Target variable

# Split the dataset into training and test sets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_tl, y_tl, random_state=42)



In [ ]:
# filter categorical cols only
categorical_cols = [colname for colname in X_train_full.columns if X_train_full[colname].dtype == "object"]

# Select numerical columns
numerical_cols = [colname for colname in X_train_full.columns if X_train_full[colname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

# Assuming you have defined your numerical_cols and categorical_cols appropriately

# Step 1: Create transformers for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# Step 2: Create a ColumnTransformer to apply different transformers to different feature types
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

In [ ]:
# # Step 2: Define the Model
# # Next, we define a random forest model with the familiar RandomForestRegressor class.
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=250, random_state=55)

In [ ]:
# Create a pipline 
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])
# This line fits the pipeline to the training data,
my_pipeline.fit(X_train, y_train)

# # Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Assuming 'preds' contains the predicted labels for the validation set, and 'y_valid' contains the true labels
precision = precision_score(y_valid, preds)
recall = recall_score(y_valid, preds)
f1 = f1_score(y_valid, preds)
conf_matrix = confusion_matrix(y_valid, preds)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:")
print(conf_matrix)


METHOD 1 RANDOM UNDERSAMPLING SCORE:
Precision: 0.6836734693877551
Recall: 0.8701298701298701
F1-score: 0.7657142857142857
Confusion Matrix:
[[63 31]
 [10 67]]

METHOD 2 RANDOM OVERSAMPLING SCCORE:
Precision: 0.9315068493150684
Recall: 0.918918918918919
F1-score: 0.9251700680272108
Confusion Matrix:
[[117  10]
 [ 12 136]]

METHOD 3 RANDOM UNDERSAMPLING WITH IMBLEARN SCORE:
Precision: 0.8023255813953488
Recall: 0.8961038961038961
F1-score: 0.8466257668711655
Confusion Matrix:
[[77 17]
 [ 8 69]]

METHOD 4 RANDOM OVERSAMPLING WITH IMBLEARN SCORE:
Precision: 0.8407643312101911
Recall: 0.9361702127659575
F1-score: 0.8859060402684564
Confusion Matrix:
[[109  25]
 [  9 132]]





Synthetic Minority Oversampling Technique (SMOTE)
This technique generates synthetic data for the minority class.

SMOTE (Synthetic Minority Oversampling Technique) works by randomly picking a point from the minority class and computing the k-nearest neighbors for this point. The synthetic points are added between the chosen point and its neighbors.

SMOTE algorithm works in 4 simple steps:

Choose a minority class as the input vector.
Find its k nearest neighbors (k_neighbors is specified as an argument in the SMOTE() function).
Choose one of these neighbors and place a synthetic point anywhere on the line joining the point under consideration and its chosen neighbor.
Repeat the steps until the data is balanced.

HOWEVER all features must be numerical.

In [ ]:

# # import library
# from imblearn.over_sampling import SMOTE

# smote = SMOTE()

# # fit predictor and target variable
# x_smote, y_smote = smote.fit_resample(x, y)

# print('Original dataset shape', Counter(y))
# print('Resample dataset shape', Counter(y_ros))

NearMiss
NearMiss is an under-sampling technique. Instead of resampling the Minority class, using a distance will make the majority class equal to the minority class.

HOWEVER all features must be numerical.

In [ ]:
# from imblearn.under_sampling import NearMiss

# nm = NearMiss()

# x_nm, y_nm = nm.fit_resample(x, y)

# print('Original dataset shape:', Counter(y))
# print('Resample dataset shape:', Counter(y_nm))